In [1]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

pd.set_option("display.max_colwidth", 80)

In [2]:
baseurl = "https://medley.life"
testurl = "https://medley.life/institutions/pref_%E5%8C%97%E6%B5%B7%E9%81%93"
testurl = "https://medley.life/institutions/5a1713039aefea1ca06cd56c/"

# user agent
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

In [3]:
res = requests.get(testurl, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")
tests = soup.find_all("li", "c-search-item")

In [4]:
address= [test.find("a").get("href") for test in tests]

In [5]:
# 詳細ページ
print(address[0])
# res = requests.get(baseurl+address[0], headers=headers)
res = requests.get(testurl, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")


/institutions/564ab1321c1b604d59c177da/


# 探索ループ

In [6]:
# ドメイン
base_url = "https://medley.life"
# target_url = "/institutions/pref_北海道/?page="

# user agent
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47.0) Gecko/20100101 Firefox/47.0",
}

prefs = ['北海道', '青森県', '岩手県', '宮城県', '秋田県', 
             '山形県', '福島県', '茨城県', '栃木県', '群馬県', 
             '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', 
             '富山県', '石川県', '福井県', '山梨県', '長野県', 
             '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', 
             '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', 
             '鳥取県', '島根県', '岡山県', '広島県', '山口県', 
             '徳島県', '香川県', '愛媛県', '高知県', '福岡県', 
             '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', 
             '鹿児島県', '沖縄県']

In [7]:
for pref in prefs:
    target_url = "/institutions/pref_"+pref+"/?page="
    # 初期化
    df_all = pd.DataFrame()
    page = 1
    last_flag = False

    while True:
        url = base_url + target_url + str(page)
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")

        if soup.find(class_="o-pagination__next") is not None or last_flag is True:
            # ページ表示
            print("--- Getting data at page = {} ---".format(page))
            time.sleep(3) # 遅延させる

            hospitals = soup.find_all("li", "c-search-item")

            # hospital内探索
            for hospital in tqdm(hospitals):
                toadd = {}

                hospital_url = hospital.find("a").get("href")
                hospital_type = hospital.find("label").get_text().strip()
                toadd["タイプ"] = hospital_type
                # time.sleep(1)
                # 病院ページにアクセス
                res = requests.get(base_url+hospital_url, headers=headers)
                soup = BeautifulSoup(res.text, "html.parser")

                rows = soup.find_all("table")[0].find_all("tr")

                for row in rows:
                    toadd[row.th.get_text()] = row.td.get_text()

                # 診療科と専門外来を追加
                for section in soup.find_all("section"):
                    if "診療科" in section.find("header").get_text().strip():
                        departments = section.find_all("a")
                        tmp = [department.get_text().strip() for department in departments]
                        toadd["診療科"] = " ".join(tmp)

                    if "専門外来" in section.find("header").get_text().strip():
                        outpatients = section.find_all("label")
                        tmp = [outpatient.get_text().strip() for outpatient in outpatients]
                        toadd["専門外来"] = " ".join(tmp)

                # データフレームに格納
                df = pd.io.json.json_normalize(toadd)
                try:
                    df_all = pd.concat([df_all, df])
                except NameError:
                    df_all = pd.io.json.json_normalize(toadd)

            page += 1

            if last_flag:
                df_all.to_csv("./"+pref+".csv")
                print("--- DONE. ---")
                break
        else:
            if last_flag is True:
                break
            else:
                last_flag = True

--- Getting data at page = 281 ---


100%|██████████| 25/25 [00:05<00:00,  4.44it/s]


--- Getting data at page = 282 ---


100%|██████████| 15/15 [00:03<00:00,  4.05it/s]

--- DONE. ---


In [183]:
# \nが気持ち悪い場合はmap
remove_n = lambda x: x.lstrip()
df_all["施設名"] = df_all["施設名"].map(remove_n)

In [9]:
df_all.to_csv("medley_test.csv")